In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from scipy import stats
import re # For more complex string operations if needed

# Set up plot display in notebook
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')


In [ ]:
# Load data from files
transaction_data_raw = pd.ExcelFile("QVI_transaction_data.xlsx")
customer_data = pd.read_csv('QVI_purchase_behaviour.csv')

# Create a copy for manipulation, keeping the original data intact
transaction_data = transaction_data_raw.parse('in')


In [ ]:
print("--- Initial transaction data information ---")
transaction_data.info()


In [ ]:
print("--- First 5 rows of transaction data ---")
transaction_data.head()


In [ ]:
print("--- Initial customer data information ---")
customer_data.info()


In [ ]:
print("--- First 5 rows of customer data ---")
customer_data.head()


In [ ]:
print("--- Converting DATE column ---")
# Convert DATE column to datetime format
transaction_data['DATE'] = pd.to_datetime(transaction_data['DATE'], origin='1899-12-30', unit='D')
print(f"Data type of DATE column after conversion: {transaction_data['DATE'].dtype}")
transaction_data['DATE'].head()


In [ ]:
print("--- Analyzing PROD_NAME ---")
# Check unique product names
print("Some product names and their counts:")
transaction_data['PROD_NAME'].value_counts().head()


In [ ]:
# Text analysis (word exploration in product names)
all_prod_words = transaction_data['PROD_NAME'].str.findall(r'\b\w+\b').explode().str.lower()
word_counts = all_prod_words.value_counts()
print("Most common words in product names:")
word_counts.head(10)


In [ ]:
# Remove Salsa products
salsa_mask = transaction_data['PROD_NAME'].str.lower().str.contains('salsa', na=False)
print(f"Number of salsa products found: {salsa_mask.sum()}")
transaction_data = transaction_data[~salsa_mask].copy()
print(f"Number of transactions remaining after removing salsa: {len(transaction_data)}")


In [ ]:
print("--- Initial summary statistics (before PROD_QTY outlier handling) ---")
transaction_data.describe(include='all')


In [ ]:
# Investigate PROD_QTY outliers
outlier_transactions_qty_200 = transaction_data[transaction_data['PROD_QTY'] == 200]
print("Transactions with PROD_QTY = 200:")
outlier_transactions_qty_200


In [ ]:
if not outlier_transactions_qty_200.empty:
    # Assume only one customer as in R documentation
    outlier_customer_id = 226000 # Based on R documentation
    customer_226000_transactions = transaction_data[transaction_data['LYLTY_CARD_NBR'] == outlier_customer_id]
    print(f"Transactions for customer LYLTY_CARD_NBR = {outlier_customer_id}:")
    customer_226000_transactions
    
    # Filter out outlier customer
    transaction_data = transaction_data[transaction_data['LYLTY_CARD_NBR'] != outlier_customer_id].copy()
    print(f"Number of transactions remaining after removing customer {outlier_customer_id}: {len(transaction_data)}")


In [ ]:
print("--- Table 1: Summary Statistics of Transaction Data (After Outlier Removal) ---")
# Set datetime_is_numeric=True to include DATE column in statistics if supported by newer pandas versions
try:
    summary_stats_post_outlier = transaction_data.describe(include='all', datetime_is_numeric=True)
except TypeError: # For older pandas versions without datetime_is_numeric
    summary_stats_post_outlier = transaction_data.describe(include='all')
summary_stats_post_outlier


In [ ]:
print("--- Transaction trends over time ---")
# Count transactions by day
transactions_by_day_counts = transaction_data.groupby('DATE').size().reset_index(name='N')
print(f"Number of days with transactions: {len(transactions_by_day_counts)}")

# Identify and handle missing dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start="2018-07-01", end="2019-06-30", freq='D')})
transactions_by_day_full = pd.merge(all_dates_df, transactions_by_day_counts, on='DATE', how='left').fillna({'N': 0})
missing_transaction_dates = transactions_by_day_full[transactions_by_day_full['N'] == 0]
print("\nDays with no transactions (e.g., Christmas):")
missing_transaction_dates.head()


In [ ]:
# Visualize transaction volume
fig, ax = plt.subplots(figsize=(12, 6))
sns.lineplot(x='DATE', y='N', data=transactions_by_day_full, ax=ax, color='dodgerblue')
ax.set_title('Number of Transactions Over Time (Overview)', fontsize=16, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Number of Transactions', fontsize=12)
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.xticks(rotation=45, ha='right')
plt.tight_layout()


In [ ]:
# Analyze December transactions
december_transactions = transactions_by_day_full[transactions_by_day_full['DATE'].dt.month == 12]
fig_dec, ax_dec = plt.subplots(figsize=(12, 6))
sns.lineplot(x='DATE', y='N', data=december_transactions, ax=ax_dec, color='tomato')
ax_dec.set_title('Number of Transactions in December', fontsize=16, fontweight='bold')
ax_dec.set_xlabel('Date', fontsize=12)
ax_dec.set_ylabel('Number of Transactions', fontsize=12)
ax_dec.xaxis.set_major_locator(mdates.DayLocator(interval=2)) # Display every 2 days
ax_dec.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.xticks(rotation=45, ha='right')
plt.tight_layout()


In [ ]:
print("--- Feature engineering ---")
# Extract the last number in the product name, assuming it's the pack size
transaction_data['PACK_SIZE'] = transaction_data['PROD_NAME'].str.findall(r'\d+').str[-1]
# Handle cases where no number is found (rare with this data)
transaction_data['PACK_SIZE'] = pd.to_numeric(transaction_data['PACK_SIZE'], errors='coerce').fillna(0).astype(int)

print("A few examples of extracted PACK_SIZE:")
transaction_data['PACK_SIZE'].head()


In [ ]:
pack_size_counts = transaction_data['PACK_SIZE'].value_counts().sort_index()
print("Distribution of PACK_SIZE:")
pack_size_counts


In [ ]:
transaction_data['PACK_SIZE'].hist(bins=len(pack_size_counts) if len(pack_size_counts) < 50 else 50, edgecolor='black')
plt.title('Distribution of Pack Size (PACK_SIZE)', fontsize=16, fontweight='bold')
plt.xlabel('Pack Size (g)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.tight_layout()


In [ ]:
# Extract initial brand (first word)
transaction_data['BRAND'] = transaction_data['PROD_NAME'].str.split().str[0].str.upper()

# Standardize brand names
brand_cleaning_map = {
    "RED": "RRD", "SNBTS": "SUNBITES", "INFZNS": "INFUZIONS",
    "WW": "WOOLWORTHS", "SMITH": "SMITHS", "NCC": "NATURAL",
    "DORITO": "DORITOS", "GRAIN": "GRNWVES", # GRNWVES could be Grain Waves
    "CC'S": "CCS" # Added CCS from R data
}
transaction_data['BRAND'] = transaction_data['BRAND'].replace(brand_cleaning_map)


In [ ]:
print("--- Table 2: Cleaned Brand Distribution ---")
cleaned_brand_counts = transaction_data['BRAND'].value_counts().sort_index()
cleaned_brand_counts.reset_index().rename(columns={'index':'BRAND', 'BRAND':'Count'})


In [ ]:
print("--- Exploring customer data ---")
# Check Structure and Summary
print("Customer data information:")
customer_data.info()


In [ ]:
print("Customer data summary statistics:")
customer_data.describe(include='all')


In [ ]:
# Check Distribution of LIFESTAGE and PREMIUM_CUSTOMER
print("LIFESTAGE distribution:")
customer_data['LIFESTAGE'].value_counts()


In [ ]:
print("PREMIUM_CUSTOMER distribution:")
customer_data['PREMIUM_CUSTOMER'].value_counts()


In [ ]:
print("--- Merging data ---")
# Perform Merge
merged_data = pd.merge(transaction_data, customer_data, on='LYLTY_CARD_NBR', how='left')
print(f"Number of rows in transaction data: {len(transaction_data)}")
print(f"Number of rows in merged data: {len(merged_data)}")


In [ ]:
# Verify Integrity
null_lifestage_count = merged_data['LIFESTAGE'].isnull().sum()
null_premium_customer_count = merged_data['PREMIUM_CUSTOMER'].isnull().sum()
print(f"Number of null values in LIFESTAGE after merge: {null_lifestage_count}")
print(f"Number of null values in PREMIUM_CUSTOMER after merge: {null_premium_customer_count}")

if null_lifestage_count == 0 and null_premium_customer_count == 0:
    print("Merge successful, no transactions missing customer information.")
else:
    print("Warning: Some transactions are missing customer information after merge.")


In [ ]:
print("--- Customer segmentation analysis ---")
sales_by_segment = merged_data.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER'], observed=True)['TOT_SALES'].sum().reset_index(name='SALES')
print("--- Table 3: Total Sales by LIFESTAGE and PREMIUM_CUSTOMER (Top 10) ---")
sales_by_segment.sort_values(by='SALES', ascending=False).head(10)


In [ ]:
sales_pivot = sales_by_segment.pivot_table(index='PREMIUM_CUSTOMER', columns='LIFESTAGE', values='SALES')
plt.figure(figsize=(14, 8))
sns.heatmap(sales_pivot, annot=True, fmt=".0f", cmap="viridis", linewidths=.5,
            cbar_kws={'label': 'Total Sales ($)'})
plt.title('Total Sales by LIFESTAGE and PREMIUM_CUSTOMER', fontsize=16, fontweight='bold')
plt.ylabel('Premium Customer Segment', fontsize=12)
plt.xlabel('Lifestage', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()


In [ ]:
num_customers_segment = merged_data.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER'], observed=True)['LYLTY_CARD_NBR'].nunique().reset_index(name='CUSTOMERS')
print("--- Table 4: Number of Customers by LIFESTAGE and PREMIUM_CUSTOMER (Top 10) ---")
num_customers_segment.sort_values(by='CUSTOMERS', ascending=False).head(10)


In [ ]:
customers_pivot = num_customers_segment.pivot_table(index='PREMIUM_CUSTOMER', columns='LIFESTAGE', values='CUSTOMERS')
plt.figure(figsize=(14, 8))
sns.heatmap(customers_pivot, annot=True, fmt=".0f", cmap="YlGnBu", linewidths=.5,
            cbar_kws={'label': 'Number of Customers'})
plt.title('Number of Customers by LIFESTAGE and PREMIUM_CUSTOMER', fontsize=16, fontweight='bold')
plt.ylabel('Premium Customer Segment', fontsize=12)
plt.xlabel('Lifestage', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()


In [ ]:
avg_units_segment = merged_data.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER'], observed=True).agg(
    total_qty=('PROD_QTY', 'sum'),
    unique_customers=('LYLTY_CARD_NBR', 'nunique')
).reset_index()
avg_units_segment['AVG_UNITS_PER_CUSTOMER'] = avg_units_segment['total_qty'] / avg_units_segment['unique_customers']
print("--- Table 5: Average Units per Customer by Segment (Top 10) ---")
avg_units_segment.sort_values(by='AVG_UNITS_PER_CUSTOMER', ascending=False).head(10)


In [ ]:
plt.figure(figsize=(12, 7))
sns.barplot(x='LIFESTAGE', y='AVG_UNITS_PER_CUSTOMER', hue='PREMIUM_CUSTOMER', data=avg_units_segment, palette='viridis', dodge=True)
plt.title('Average Units per Customer by Segment', fontsize=16, fontweight='bold')
plt.xlabel('Lifestage', fontsize=12)
plt.ylabel('Avg. Units / Customer', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.legend(title='Customer Segment')
plt.tight_layout()


In [ ]:
avg_price_segment = merged_data.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER'], observed=True).agg(
    total_sales_val=('TOT_SALES', 'sum'),
    total_qty_val=('PROD_QTY', 'sum')
).reset_index()
avg_price_segment['AVG_PRICE_PER_UNIT'] = avg_price_segment['total_sales_val'] / avg_price_segment['total_qty_val']
print("--- Table 6: Average Price per Unit by Segment (Top 10) ---")
avg_price_segment.sort_values(by='AVG_PRICE_PER_UNIT', ascending=False).head(10)


In [ ]:
plt.figure(figsize=(12, 7))
sns.barplot(x='LIFESTAGE', y='AVG_PRICE_PER_UNIT', hue='PREMIUM_CUSTOMER', data=avg_price_segment, palette='coolwarm', dodge=True)
plt.title('Average Price per Unit by Segment', fontsize=16, fontweight='bold')
plt.xlabel('Lifestage', fontsize=12)
plt.ylabel('Avg. Price / Unit ($)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.legend(title='Customer Segment')
plt.tight_layout()


In [ ]:
# Calculate price per unit for each transaction
merged_data['PRICE_PER_UNIT'] = merged_data['TOT_SALES'] / merged_data['PROD_QTY']

# Define groups for t-test
group_mainstream = merged_data[
    (merged_data['LIFESTAGE'].isin(["YOUNG SINGLES/COUPLES", "MIDAGE SINGLES/COUPLES"])) &
    (merged_data['PREMIUM_CUSTOMER'] == "Mainstream")
]['PRICE_PER_UNIT'].dropna()

group_other = merged_data[
    (merged_data['LIFESTAGE'].isin(["YOUNG SINGLES/COUPLES", "MIDAGE SINGLES/COUPLES"])) &
    (merged_data['PREMIUM_CUSTOMER'].isin(["Budget", "Premium"])) # Includes Budget and Premium
]['PRICE_PER_UNIT'].dropna()


In [ ]:
if not group_mainstream.empty and not group_other.empty:
    t_stat, p_value = stats.ttest_ind(group_mainstream, group_other,
                                      equal_var=False, # Welch's t-test
                                      alternative='greater') # Test if mainstream is greater
    print("--- Table 7: T-test Results for Price Per Unit ---")
    print(f"Comparison: Mainstream (Young/Midage Singles/Couples) vs. Budget/Premium (Young/Midage Singles/Couples)")
    print(f"T-statistic: {t_stat:.4f}")
    print(f"P-value: {p_value:.4f}") # Format p-value for readability
    print(f"Avg Price (Mainstream): {group_mainstream.mean():.4f}")
    print(f"Avg Price (Budget/Premium): {group_other.mean():.4f}")
    if p_value < 0.05:
        print("Conclusion: There is statistical evidence that the Mainstream group pays a significantly higher price per unit.")
    else:
        print("Conclusion: There is not enough statistical evidence that the Mainstream group pays a significantly higher price per unit.")
else:
    print("Not enough data to perform T-test for the selected groups.")


In [ ]:
print("--- Deep dive: Mainstream, Young Singles/Couples ---")
target_lifestage = "YOUNG SINGLES/COUPLES"
target_premium = "Mainstream"

# Filter target segment
segment1 = merged_data[
    (merged_data['LIFESTAGE'] == target_lifestage) &
    (merged_data['PREMIUM_CUSTOMER'] == target_premium)
].copy()

# Other segments (all customers not in target segment)
other_segments = merged_data[
    ~((merged_data['LIFESTAGE'] == target_lifestage) &
      (merged_data['PREMIUM_CUSTOMER'] == target_premium))
].copy()


In [ ]:
if not segment1.empty and not other_segments.empty:
    # Calculate total quantity for each group
    quantity_segment1_total = segment1['PROD_QTY'].sum()
    quantity_other_total = other_segments['PROD_QTY'].sum()
    
    # Calculate proportion of each brand in total quantity
    quantity_segment1_by_brand = segment1.groupby('BRAND', observed=True)['PROD_QTY'].sum() / quantity_segment1_total
    quantity_other_by_brand = other_segments.groupby('BRAND', observed=True)['PROD_QTY'].sum() / quantity_other_total
    
    # Calculate brand affinity
    brand_affinity = pd.merge(
        quantity_segment1_by_brand.rename('targetSegment'),
        quantity_other_by_brand.rename('other'),
        on='BRAND', how='outer'
    ).fillna(0)
    brand_affinity['affinityToBrand'] = brand_affinity['targetSegment'] / brand_affinity['other']
    brand_affinity.replace([float('inf'), -float('inf')], pd.NA, inplace=True) # Handle division by zero
    brand_affinity.dropna(subset=['affinityToBrand'], inplace=True) # Remove NA due to division by zero
    
    print("--- Table 8: Brand Affinity of 'Mainstream, Young Singles/Couples' Segment (Top 10) ---")
    brand_affinity.sort_values(by='affinityToBrand', ascending=False).head(10)


In [ ]:
if not segment1.empty and not other_segments.empty:
    # Calculate proportion of each pack size in total quantity
    quantity_segment1_by_pack = segment1.groupby('PACK_SIZE')['PROD_QTY'].sum() / quantity_segment1_total
    quantity_other_by_pack = other_segments.groupby('PACK_SIZE')['PROD_QTY'].sum() / quantity_other_total
    
    # Calculate pack size affinity
    pack_affinity = pd.merge(
        quantity_segment1_by_pack.rename('targetSegment'),
        quantity_other_by_pack.rename('other'),
        on='PACK_SIZE', how='outer'
    ).fillna(0)
    pack_affinity['affinityToPack'] = pack_affinity['targetSegment'] / pack_affinity['other']
    pack_affinity.replace([float('inf'), -float('inf')], pd.NA, inplace=True)
    pack_affinity.dropna(subset=['affinityToPack'], inplace=True)
    
    print("--- Table 9: Pack Size Preference of 'Mainstream, Young Singles/Couples' Segment (Top 10) ---")
    pack_affinity.sort_values(by='affinityToPack', ascending=False).head(10)


In [ ]:
if not segment1.empty and not other_segments.empty:
    # Investigate brands for high affinity pack size
    high_affinity_pack_size = 270 # Based on results from R documentation
    if high_affinity_pack_size in pack_affinity.index: # Check if pack size exists in affinity results
        brands_for_pack_size_270 = merged_data[merged_data['PACK_SIZE'] == high_affinity_pack_size]['BRAND'].unique()
        print(f"Products with {high_affinity_pack_size}g pack size: {', '.join(brands_for_pack_size_270)}")
    else:
        # Fallback if 270g is not the top, pick the actual top from calculation
        if not pack_affinity.empty:
            top_pack_size_from_py = pack_affinity.sort_values(by='affinityToPack', ascending=False).index[0]
            brands_for_top_pack_size = merged_data[merged_data['PACK_SIZE'] == top_pack_size_from_py]['BRAND'].unique()
            print(f"Products with highest affinity pack size ({top_pack_size_from_py}g): {', '.join(brands_for_top_pack_size)}")
else:
    print("Not enough data in segment1 or other_segments to perform deep dive analysis.")
